In [1]:
from data_loader import OpenImagesDataset
from model_utils import device
from model_transformations import Transformations
from torch.utils.data import DataLoader
from params import NUM_EPOCHS, GRID_SIZE,IMAGE_SIZE, NUM_ANCHOR_BOXES, BATCH_SIZE
import torch.optim as optim
import torch
from model_loss import YoloLoss
from tqdm import tqdm
from model import DogDetectorModel
import numpy as np
from model_decoder import Decoder
from non_max_surpression import NonMaxSurpression
from true_box_extractor import true_box_extractor
from mean_average_precision import MAP

In [2]:
# Reading in the training data
trainingData = OpenImagesDataset(rootDirectory='open-images-v6', 
                                 anchorBoxes='centroids.npy', 
                                 transform=Transformations, 
                                 dataType='train', 
                                 gridSize=GRID_SIZE, 
                                 imageSize=IMAGE_SIZE)    

# Defining the training data
trainDataLoader = DataLoader(dataset=trainingData, 
                             batch_size=BATCH_SIZE,
                             num_workers=4,
                             shuffle=False)

# Reading in the training data
validationData = OpenImagesDataset(rootDirectory='open-images-v6', 
                                 anchorBoxes='centroids.npy', 
                                 transform=Transformations, 
                                 dataType='validation', 
                                 gridSize=GRID_SIZE, 
                                 imageSize=IMAGE_SIZE)    

# Defining the training data
validationDataLoader = DataLoader(dataset=trainingData, 
                             batch_size=BATCH_SIZE,
                             num_workers=4,
                             shuffle=True)

In [3]:
# Initializing the model
model = DogDetectorModel(gridSize=13, numAnchorBoxes=NUM_ANCHOR_BOXES)

In [4]:
# Moving the model to the GPU
model = model.to(device)

In [5]:
# Defining the optimizer
optimizer = optim.SGD(list(model.parameters()), lr=0.00001, momentum=0.9, weight_decay=0.0005)

In [6]:
lossYolo = YoloLoss('centroids.npy', device)

In [ ]:
modelInput, label = trainingData.__getitem__(15)

In [ ]:
modelInput = modelInput.reshape((1,3,416,416))

In [ ]:
label = label.reshape((1,13,13,35))

In [ ]:
# loop =  tqdm(enumerate(trainDataLoader), total=len(trainDataLoader), leave=True)
# for batchIndex, (modelInput, label) in loop:
#     break

In [ ]:
# Initializing the non max supressor for predictions
nonMaxSurpressionPredicted = NonMaxSurpression()

# Initializing the docoder
decoder = Decoder('centroids.npy', device)

In [ ]:
for epoch in range(0, NUM_EPOCHS):    
    # ****************** TRAINING ******************
    
    # Setting the model to training
    model.train()
    
    # Defining list to hold the mean loss
    meanLoss = []
    
    # Defining lists to hold all the predictions and ground truths for all elements in each batch
    predictedBoxesAllBatches = []
    trueBoxesAllBatches = []
    
    # Defining loop to get the nice progress bar
    loop =  tqdm(enumerate(trainDataLoader), total=len(trainDataLoader), leave=True)
    
    for batchIndex, (modelInput, label) in loop:

        # zero the parameter gradients
        optimizer.zero_grad()

        # Moving the model input/label to GPU 
        modelInput = modelInput.to(device)
        label = label.to(device)
        
        # Computing the model output
        output = model(modelInput)

        # Computing the loss
        loss = YoloLoss(output, label)
        
        # Backpropogating the error
        loss.backward()
        
        # Executing gradient descent
        optimizer.step()

        # Appending the loss to the list
        meanLoss.append(loss.item())

        with torch.no_grad():
            # Decoding the labels
            decodedLabel = decoder(torch.clone(label), 'label')
            
            # Decoding the output
            decodedOutput = decoder(torch.clone(output))    
            
            # Executing non max surpression on the decoded output to get the object detectiosn
            predictedBoxes = nonMaxSurpressionPredicted(decodedOutput)
            
            # Extracting the true bound boxes from the decoded label
            trueBoxes = true_box_extractor(label, decodedLabel)
            
            # Adding the detected boxes onto the list
            predictedBoxesAllBatches += predictedBoxes
            
            # Adding the true boxes onto the list
            trueBoxesAllBatches += trueBoxes
    
    # Updating the output
    loop.set_description(f"Epoch: [{epoch+1}/{NUM_EPOCHS}]")     
    loop.set_postfix(loss=loss.item())
        
        
    # Computing the MAP value
    mapValue = MAP(predictedBoxesAllBatches.copy(), trueBoxesAllBatches.copy())    
        
    print(f"====> Train Mean Loss: {sum(meanLoss)/len(meanLoss)}")
    print(f"====> Train MAP: {mapValue}")
                
    # ****************** VALIDATION ******************
    
    # Setting the model to evaluation mode
    model.eval()
    
    # Resetting lists from above
    meanLoss = []
    predictedBoxesAllBatches = []
    trueBoxesAllBatches = []
    
    # Turning of the gradient
    with torch.no_grad():
        
        # Defining loop to get the nice progress bar for the validation data
        loop =  tqdm(enumerate(validationDataLoader), total=len(validationDataLoader), leave=True)
        
        # Iterating through the batches of the validation data
        for batchIndex, [modelInput, label] in loop:

            # Moving the model input/label to GPU 
            modelInput = modelInput.to(device)
            label = label.to(device)

            # Computing the model output
            output = model(modelInput)
            
            # Computing the loss
            loss = YoloLoss(output, label)
            
            # Appending the loss
            meanLoss.append(loss.item())
            
            # Decoding the labels
            decodedLabel = decoder(torch.clone(label), 'label')
            
            # Decoding the output
            decodedOutput = decoder(torch.clone(output))    
            
            # Executing non max surpression on the decoded output to get the object detectiosn
            predictedBoxes = nonMaxSurpressionPredicted(decodedOutput)
            
            # Extracting the true bound boxes from the decoded label
            trueBoxes = true_box_extractor(label, decodedLabel)
            
            # Adding the detected boxes onto the list
            predictedBoxesAllBatches += predictedBoxes
            
            # Adding the true boxes onto the list
            trueBoxesAllBatches += trueBoxes
            
        loop.set_description(f"Epoch: [{epoch+1}/{NUM_EPOCHS}]")     
        loop.set_postfix(loss=loss.item())
            
        # Computing the MAP value
        mapValue = MAP(predictedBoxesAllBatches.copy(), trueBoxesAllBatches.copy())    
            
        print(f"====> Validation Loss: {sum(meanLoss)/len(meanLoss)}")
        print(f"====> Validation MAP: {mapValue}")
            


        
        
    

In [ ]:
from model_utils import plot_tensor

In [ ]:
plot_tensor(modelInput[0].cpu(), decodedOutput[0][ 5,  7, 20:25].reshape((1,-1)).numpy())